# Knowledge Distillation

**Module 03 | Notebook 2 of 5**

Knowledge distillation trains a smaller "student" model to mimic a larger "teacher" model.

## Learning Objectives

By the end of this notebook, you will be able to:
1. Understand the teacher-student paradigm
2. Implement KL divergence loss for distillation
3. Train a distilled model from scratch
4. Compare performance before and after distillation

---

In [ ]:
%%capture
!pip install transformers datasets accelerate evaluate torch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    AutoConfig, get_scheduler
)
from datasets import load_dataset
from torch.utils.data import DataLoader
import evaluate
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

---

## How Distillation Works

### Traditional Training vs. Distillation

```
TRADITIONAL TRAINING:
┌───────────┐     Hard Labels       ┌───────────┐
│   Input   │────────────────────→  │  Student  │  Loss: CrossEntropy
└───────────┘     [0, 1, 0]         └───────────┘

DISTILLATION:
┌───────────┐                        
│   Input   │                        
└─────┬─────┘                        
      │                              
      ├──────────→ ┌───────────┐     Soft Labels (probabilities)
      │            │  Teacher  │────→ [0.05, 0.90, 0.05]
      │            └───────────┘            │
      │                                     │ KL Divergence
      │                                     ▼
      └──────────→ ┌───────────┐     ────────────────
                   │  Student  │────→ [0.10, 0.85, 0.05]
                   └───────────┘
```

### Why Soft Labels?

Soft labels contain **more information** than hard labels:

| Label Type | Example | Information Content |
|------------|---------|--------------------|
| Hard | [0, 1, 0] ("cat") | Only the correct class |
| Soft | [0.05, 0.90, 0.05] | "Mostly cat, slightly similar to dog and bird" |

### The Distillation Loss

```
L_total = α × L_distill + (1-α) × L_student

Where:
  L_distill = KL(teacher_soft_logits, student_soft_logits) × T²
  L_student = CrossEntropy(student_logits, hard_labels)
  T = Temperature (higher = softer probabilities)
  α = Weighting factor (typically 0.5-0.9)
```

---

## Load Teacher and Student Models

In [ ]:
# Teacher: A fine-tuned BERT model
teacher_name = "distilbert-base-uncased-finetuned-sst-2-english"
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_name)
teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_name).to(device)
teacher_model.eval()  # Teacher is always in eval mode

teacher_params = sum(p.numel() for p in teacher_model.parameters())
print(f"Teacher: {teacher_name}")
print(f"  Parameters: {teacher_params:,}")

In [ ]:
# Student: A smaller model (we'll create a smaller DistilBERT)
# For real distillation, you'd use an even smaller architecture
# Here we'll use the same architecture but show the training process

student_name = "distilbert-base-uncased"  # Base model, not fine-tuned
student_tokenizer = AutoTokenizer.from_pretrained(student_name)
student_model = AutoModelForSequenceClassification.from_pretrained(
    student_name,
    num_labels=2
).to(device)

student_params = sum(p.numel() for p in student_model.parameters())
print(f"\nStudent: {student_name}")
print(f"  Parameters: {student_params:,}")

---

## Prepare the Dataset

In [ ]:
# Load SST-2 dataset (same as teacher was trained on)
dataset = load_dataset("glue", "sst2")

print("Dataset:")
print(f"  Train: {len(dataset['train'])} examples")
print(f"  Validation: {len(dataset['validation'])} examples")

In [ ]:
# Use subset for faster training
train_size = 2000
val_size = 500

train_data = dataset['train'].shuffle(seed=42).select(range(train_size))
val_data = dataset['validation'].shuffle(seed=42).select(range(val_size))

# Tokenization
def tokenize_function(examples):
    return teacher_tokenizer(
        examples['sentence'],
        padding='max_length',
        truncation=True,
        max_length=128
    )

train_tokenized = train_data.map(tokenize_function, batched=True)
val_tokenized = val_data.map(tokenize_function, batched=True)

# Set format for PyTorch
train_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Create dataloaders
train_loader = DataLoader(train_tokenized, batch_size=16, shuffle=True)
val_loader = DataLoader(val_tokenized, batch_size=32)

---

## Implement Distillation Loss

In [ ]:
class DistillationLoss(nn.Module):
    """
    Distillation loss combining soft label matching and hard label classification.
    
    L_total = α × L_distill + (1-α) × L_student
    """
    def __init__(self, temperature=4.0, alpha=0.7):
        super().__init__()
        self.temperature = temperature
        self.alpha = alpha
        self.ce_loss = nn.CrossEntropyLoss()
        self.kl_loss = nn.KLDivLoss(reduction='batchmean')
    
    def forward(self, student_logits, teacher_logits, labels):
        # Soft labels from teacher and student
        teacher_soft = F.softmax(teacher_logits / self.temperature, dim=-1)
        student_soft = F.log_softmax(student_logits / self.temperature, dim=-1)
        
        # Distillation loss (KL divergence)
        distill_loss = self.kl_loss(student_soft, teacher_soft) * (self.temperature ** 2)
        
        # Student loss (cross-entropy with hard labels)
        student_loss = self.ce_loss(student_logits, labels)
        
        # Combined loss
        total_loss = self.alpha * distill_loss + (1 - self.alpha) * student_loss
        
        return total_loss, distill_loss, student_loss

# Initialize loss function
distill_criterion = DistillationLoss(temperature=4.0, alpha=0.7)
print("Distillation loss initialized with:")
print(f"  Temperature: {distill_criterion.temperature}")
print(f"  Alpha: {distill_criterion.alpha}")

### Understanding Temperature

Temperature controls how "soft" the probability distribution is:

| Temperature | Effect |
|-------------|--------|
| T = 1 | Original distribution |
| T = 2-5 | Smoother distribution (recommended) |
| T > 10 | Very uniform distribution |

In [ ]:
# Visualize temperature effect
import matplotlib.pyplot as plt

logits = torch.tensor([2.0, 0.5, -1.0])
temps = [1, 2, 4, 8]

fig, axes = plt.subplots(1, 4, figsize=(14, 3))

for ax, T in zip(axes, temps):
    probs = F.softmax(logits / T, dim=-1).numpy()
    ax.bar(['Class A', 'Class B', 'Class C'], probs, color=['#3498db', '#2ecc71', '#e74c3c'])
    ax.set_ylim(0, 1)
    ax.set_title(f'Temperature = {T}')
    ax.set_ylabel('Probability')

plt.suptitle('Effect of Temperature on Softmax Probabilities', fontsize=12)
plt.tight_layout()
plt.show()

---

## Train the Student Model

In [ ]:
# Optimizer and scheduler
optimizer = AdamW(student_model.parameters(), lr=2e-5, weight_decay=0.01)
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)

lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps
)

In [ ]:
def train_epoch(student_model, teacher_model, train_loader, optimizer, scheduler, criterion, device):
    """Train for one epoch."""
    student_model.train()
    total_loss = 0
    total_distill_loss = 0
    total_student_loss = 0
    
    progress_bar = tqdm(train_loader, desc="Training")
    
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        # Get teacher predictions (no gradient)
        with torch.no_grad():
            teacher_outputs = teacher_model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            teacher_logits = teacher_outputs.logits
        
        # Get student predictions
        student_outputs = student_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        student_logits = student_outputs.logits
        
        # Calculate distillation loss
        loss, distill_loss, student_loss = criterion(
            student_logits, teacher_logits, labels
        )
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        total_distill_loss += distill_loss.item()
        total_student_loss += student_loss.item()
        
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'distill': f'{distill_loss.item():.4f}'
        })
    
    return {
        'total_loss': total_loss / len(train_loader),
        'distill_loss': total_distill_loss / len(train_loader),
        'student_loss': total_student_loss / len(train_loader)
    }

In [ ]:
def evaluate_model(model, val_loader, device):
    """Evaluate model accuracy."""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            predictions = torch.argmax(outputs.logits, dim=-1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    
    return correct / total

In [ ]:
# Evaluate teacher first (baseline)
teacher_acc = evaluate_model(teacher_model, val_loader, device)
print(f"Teacher Accuracy: {teacher_acc:.2%}")

# Evaluate un-distilled student
student_acc_before = evaluate_model(student_model, val_loader, device)
print(f"Student Accuracy (before distillation): {student_acc_before:.2%}")

In [ ]:
# Training loop
print("\n" + "="*50)
print("Starting Distillation Training")
print("="*50)

history = []

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    print("-" * 30)
    
    # Train
    train_metrics = train_epoch(
        student_model, teacher_model, train_loader,
        optimizer, lr_scheduler, distill_criterion, device
    )
    
    # Evaluate
    student_acc = evaluate_model(student_model, val_loader, device)
    
    history.append({
        'epoch': epoch + 1,
        'loss': train_metrics['total_loss'],
        'accuracy': student_acc
    })
    
    print(f"Loss: {train_metrics['total_loss']:.4f}")
    print(f"  - Distillation Loss: {train_metrics['distill_loss']:.4f}")
    print(f"  - Student Loss: {train_metrics['student_loss']:.4f}")
    print(f"Validation Accuracy: {student_acc:.2%}")

print("\n" + "="*50)
print("Distillation Complete!")
print("="*50)

---

## Compare Results

In [ ]:
# Final evaluation
student_acc_after = evaluate_model(student_model, val_loader, device)

print("\n" + "="*50)
print("Results Comparison")
print("="*50)
print(f"\nTeacher Accuracy:                    {teacher_acc:.2%}")
print(f"Student Accuracy (before):           {student_acc_before:.2%}")
print(f"Student Accuracy (after distill):    {student_acc_after:.2%}")
print(f"\nImprovement:                         +{(student_acc_after - student_acc_before)*100:.1f}%")
print(f"Gap to Teacher:                      {(teacher_acc - student_acc_after)*100:.1f}%")

In [ ]:
# Visualize training progress
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

epochs = [h['epoch'] for h in history]
losses = [h['loss'] for h in history]
accuracies = [h['accuracy'] for h in history]

# Loss plot
axes[0].plot(epochs, losses, 'b-o', linewidth=2, markersize=8)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

# Accuracy plot
axes[1].plot(epochs, accuracies, 'g-o', linewidth=2, markersize=8, label='Student')
axes[1].axhline(y=teacher_acc, color='r', linestyle='--', label=f'Teacher ({teacher_acc:.2%})')
axes[1].axhline(y=student_acc_before, color='gray', linestyle=':', label=f'Student (before)')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Validation Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## Test the Distilled Model

In [ ]:
# Test on new examples
test_texts = [
    "This movie was absolutely wonderful!",
    "Terrible waste of time.",
    "The acting was superb but the plot was confusing."
]

print("Comparison on Test Examples:")
print("="*60)

for text in test_texts:
    inputs = teacher_tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        teacher_out = teacher_model(**inputs).logits
        student_out = student_model(**inputs).logits
    
    teacher_probs = F.softmax(teacher_out, dim=-1)[0]
    student_probs = F.softmax(student_out, dim=-1)[0]
    
    teacher_pred = "POSITIVE" if teacher_probs[1] > 0.5 else "NEGATIVE"
    student_pred = "POSITIVE" if student_probs[1] > 0.5 else "NEGATIVE"
    
    print(f"\nText: {text}")
    print(f"  Teacher: {teacher_pred} ({teacher_probs[1]:.2%} positive)")
    print(f"  Student: {student_pred} ({student_probs[1]:.2%} positive)")

---

## 🎯 Student Challenge

### Challenge: Experiment with Hyperparameters

In [ ]:
# TODO: Experiment with different distillation settings:

# 1. Try different temperatures (T = 1, 2, 8, 16)
#    - Record accuracy for each
#    - Which temperature works best?

# 2. Try different alpha values (α = 0.3, 0.5, 0.7, 0.9)
#    - α closer to 1 = more weight on distillation loss
#    - α closer to 0 = more weight on hard labels

# 3. What happens with more training epochs?

# Your solution:


---

## Key Takeaways

1. **Distillation** transfers knowledge from a large model to a smaller one
2. **Soft labels** contain more information than hard labels
3. **Temperature** controls how soft the probability distribution is
4. **Combined loss** uses both distillation and classification objectives
5. Student can achieve **95-99%** of teacher performance

---

## Next Steps

Continue to `03_pruning.ipynb` for weight pruning techniques!